In [186]:
%matplotlib inline
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import copy
import cvxopt as opt
from cvxopt import blas, solvers


np.random.seed(123)

# Turn off progress printing 
solvers.options['show_progress'] = False



In [224]:
#Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.
tickers = ['AAPL', 'MSFT', 'T', 'GOOG']
nAssets = len(tickers)
print("Number of assets: %d"%nAssets)

# Define which online source one should use
data_source = 'yahoo'

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2007-01-02'
end_date = '2007-12-31'

# User pandas_reader.data.DataReader to load the desired data. As simple as that.
#trainingData = data.DataReader(tickers, data_source, start_date, end_date)['Adj Close']
trainingData = data.get_data_yahoo(tickers, start_date, end_date, interval='m')

# Getting just the adjusted closing prices. This will return a Pandas DataFrame
# The index in this DataFrame is the major index of the panel_data.
close = trainingData.ix['Adj Close']
close = close.fillna(method='ffill')

# Opening prices.
start = trainingData.ix['Open']
start = start.fillna(method='ffill')

Number of assets: 4


/usr/local/lib/python2.7/dist-packages/urllib3/util/ssl_.py:137: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning
/usr/local/lib/python2.7/dist-packages/urllib3/util/ssl_.py:137: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning


<class 'pandas.core.panel.Panel'>
Dimensions: 6 (items) x 12 (major_axis) x 4 (minor_axis)
Items axis: Adj Close to Volume
Major_axis axis: 2007-12-01 00:00:00 to 2007-01-01 00:00:00
Minor_axis axis: AAPL to T


In [225]:
def calculateReturns():
    X0 = [start[ticker].iloc[-1] for ticker in tickers]
    print("Starting prices: ", X0)

    Returns = []
    # Closing price every day.
    for i, ticker in enumerate(tickers):
        closingPrices = close[ticker]
        # Caclulate returns
        calcReturn = lambda x: float(x - X0[i]) / X0[i]
        Returns.append(closingPrices.apply(calcReturn))
    return Returns
    
def optimalPortfolio(Returns):
    # Calculate the expected Return for each asset.
    n = len(Returns )
    print(n)
    mus = np.mean(np.asarray(Returns), axis=1)
    mus = np.reshape(mus, (2,1))
    Cov = opt.matrix(np.cov(Returns))
    Mus = opt.matrix(mus)
       
    q = opt.matrix(np.zeros((n, 1)))
    
    # Create constraint matrices
    retMin = 1.4
    # w >= 0 and min return constraint.
    G = -opt.matrix(np.concatenate((mus.T, np.eye(n))))   # negative n x n identity matrix
    h = opt.matrix(np.concatenate((np.ones((1,1))*retMin ,np.zeros((n ,1)))))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    portfolio = solvers.qp(Cov, q, G, h, A, b)
    return (portfolio, Mus)
returns = calculateReturns()
print(returns)
optimalPortfolio, Mus = optimalPortfolio(returns)

optimalPortfolio = optimalPortfolio['x']
print(optimalPortfolio)


('Starting prices: ', [12.327143, 29.91, 35.669998, 231.49435400000002])
[Date
2007-12-01    1.057776
2007-11-01    0.893013
2007-10-01    0.973317
2007-09-01    0.594341
2007-08-01    0.438615
2007-07-01    0.368803
2007-06-01    0.267826
2007-05-01    0.258996
2007-04-01    0.036784
2007-03-01   -0.034794
2007-02-01   -0.121019
2007-01-01   -0.109384
Freq: -1MS, Name: AAPL, dtype: float64, Date
2007-12-01   -0.067479
2007-11-01   -0.122768
2007-10-01   -0.038961
2007-09-01   -0.230855
2007-08-01   -0.252534
2007-07-01   -0.245770
2007-06-01   -0.233282
2007-05-01   -0.204119
2007-04-01   -0.223569
2007-03-01   -0.277250
2007-02-01   -0.271995
2007-01-01   -0.202476
Freq: -1MS, Name: MSFT, dtype: float64, Date
2007-12-01   -0.321192
2007-11-01   -0.375909
2007-10-01   -0.323206
2007-09-01   -0.314785
2007-08-01   -0.354301
2007-07-01   -0.371296
2007-06-01   -0.333728
2007-05-01   -0.336297
2007-04-01   -0.383986
2007-03-01   -0.372690
2007-02-01   -0.414532
2007-01-01   -0.407585
Fre

ValueError: cannot reshape array of size 4 into shape (2,1)

In [189]:
returns = blas.dot(Mus, optimalPortfolio)
print(returns)

0.0018328223675


In [203]:
def calc_kd(close_data):
    K = [[50 for _ in tickers]]
    D = [[50 for _ in tickers]]
    low = [np.inf for _ in tickers]
    high = [0 for _ in tickers]
    ind=close_data.shape[0] - 1
    
    while ind > 0:
        row = close_data.iloc[ind]
        for t in range(n_ticks):
            if row[t] < low[t]:
                low[t] = row[t]
            if row[t] > high[t]:
                high[t] = row[t] 

        if row.name.is_month_end:
            # calculate indices
            k_list = [0 for _ in tickers]
            d_list = [0 for i in tickers]
            for t in range(n_ticks):
                rsv = ((row[t] - low[t])*100.0) / (high[t] - low[t] + 0.1)
                k = rsv*(1.0/3) + K[-1][t]*(2.0/3)

                k_list[t]  = k				
                d = k*(1.0/3) + D[-1][t]*(2.0/3)
#				print('d = ', d)
                d_list[t]= d
		  
#			k_list = np.array(k_list)
#			d_list = np.array(d_list)
			
            K.append(copy.copy(k_list))
            D.append(copy.copy(d_list))
			
            low = [np.inf for _ in tickers]
            high = [0 for _ in tickers]
        ind -= 1
  
    return K,D
	
K,D = calc_kd(close)
print(K)

[[50, 50], [65.625929183434238, 50.000016025625648], [60.029153649029325, 33.33334401708376], [72.298393916814177, 35.613795549278777]]


In [ ]:
def conservativeStrategy():
    

In [ ]:
def getTestData():
    start_date = '2008-01-02'
    end_date = '2008-12-31'
    testData = data.get_data_yahoo(tickers, start_date, end_date, interval='m')

    # Getting just the adjusted closing prices. This will return a Pandas DataFrame
    # The index in this DataFrame is the major index of the panel_data.
    close = testData.ix['Adj Close']
    close = close.fillna(method='ffill')

    # Opening prices.
    start = testData.ix['Open']
    start = start.fillna(method='ffill')
    return (start, close)